# **Time Series synthetic weather data generation using TimeGan Model**

* TimeGAN - Utilizing this model from the paper and github
* Following along with the TimeGAN_Synthetic_stock_data.ipynb which generates synthetic stock data.


**Dataset Information**
* The data used in this notebook was downloaded from ESFGR and has been preprocessed to for simplicity
* Preprocessing steps:
  - Downloaded as a NetCDF file (.nc) from ESFGR with only one variable included: temperature (tas).
  -  Data is output from global climate model (GCMs) with a daily 100km resolution. GCMs are what are of interest for meterologists to have downscaled data from.
  -  365 rows containing the daily average temperature (tas) from 01/01/2015 to 12/31/2015
  -  There are two columns: Date and tas (might need to get rid of the date one)
  -  **Need to standardize the data**


In [59]:
#Import the necessary libraries

import ydata_synthetic
from os import path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

#Below is giving errors for some reason
from ydata_synthetic.synthesizers import ModelParameters, TrainParameters
from ydata_synthetic.preprocessing.timeseries.stock import transformations

#from ydata_synthetic.preprocessing.timeseries import processed_stock
from ydata_synthetic.synthesizers.timeseries import TimeSeriesSynthesizer

#TypeError: unsupported operand type(s) for |: 'types.GenericAlias' and 'NoneType'

TypeError: unsupported operand type(s) for |: 'types.GenericAlias' and 'NoneType'

**Loading dataset and standardizing**
* (0) MinMaxScaler: Min Max normalizer. Function taken from [data_loading.py](https://github.com/jsyoon0823/TimeGAN/blob/master/data_loading.py) from TimeGAN.
* (1) real_data_loading: Load and preprocess real data. Function taken and edited from [data_loading.py](https://github.com/jsyoon0823/TimeGAN/blob/master/data_loading.py) from TimeGAN.


In [54]:
#Loading dataset and standardizing
#Read in the dataset and set index

weather_df = pd.read_csv('/Users/gabbyvaillant/Downloads/BNL/temperature_BNL_tas_2015.csv')
weather_df = weather_df.set_index('Date').sort_index()

#Using the transformations function to normalize the data and get into sequences

data_path = '/Users/gabbyvaillant/Downloads/BNL/temperature_BNL_tas_2015.csv'
processed_weather = transformations(data_path, seq_len=7)

cols = list(weather_df.columns)
print(weather_df.shape)

#NOTE: when using the real_data_loading function that is called inside of the transformations
#function, it reverses the order of the data. I think it does this because the example uses stock
#data which begins with the first row being the most recent stock price. Our weather data is 
#already in chronological order so when we call this function, it will be backwards.
#BUT i dont think this matters because no matter the input order, it will randomly shuffle
#the data to ensure it is i.i.d

(365, 1)


**Defining Model hyperparameters**

**Networks:**
- Generator
- Discriminator
- Embedder
- Recovery Network

The parameters should be optimized and tailored to the specific dataset you are working with.

In [55]:
#Specific parameters for TimeGAN model

seq_len = 7 #ask Tianqiao how to choose this? 7 days?

#ORIGINAL STUFF:
n_seq = 6
hidden_dim = 24
gamma = 1

noise_dim = 32
dim = 128
batch_size = 128

log_step = 100
learning_rate = 5e-4
# For quick prototyping
# epochs=50000
epochs = 10

gan_args = ModelParameters(
    batch_size=batch_size, lr=learning_rate, noise_dim=noise_dim, layers_dim=dim
)

train_args = TrainParameters(
    epochs=epochs, sequence_length=seq_len, number_sequences=n_seq
)

**Training the GAN synthetizer**


In [58]:
if path.exists("synthesizer_stock.pkl"):
    synth = TimeSeriesSynthesizer.load("synthesizer_weather.pkl")
else:
    synth = TimeSeriesSynthesizer(modelname="timegan", model_parameters=gan_args)
    synth.fit(weather_df, train_args, num_cols=cols)
    synth.save("synthesizer_weather.pkl")

NameError: name 'TimeSeriesSynthesizer' is not defined